# Imports
Ici on peut retrouver les importations nécéssaires pour le projet
Marche à suivre pour sklearn: https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

In [1]:
import nltk
# get punkt and perceptron average
nltk.download("averaged_perceptron_tagger")
nltk.download("stopwords")
nltk.download("punkt")
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\WickkaWizz\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\WickkaWizz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\WickkaWizz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Feature Extraction
Dans cette section nous allons extraire les features du texte en utilisant nltk pour générer les stop-words que nous allons enlever du texte, en faisant le stemming des données avec Porter, en faisant le POS tagging et pour terminer nous allons aussi établir une grammaire pour pouvoir voir la structure du texte.

## Porter
Ici nous avons le stemmer Porter et nous faisons quelques tests pour vérifier qu'il fonctionne

In [2]:
# https://www.datacamp.com/tutorial/stemming-lemmatization-python
porter = PorterStemmer()
print("Porter Stemmer")
print(porter.stem("cats"))
print(porter.stem("trouble"))
print(porter.stem("troubling"))
print(porter.stem("troubled"))

Porter Stemmer
cat
troubl
troubl
troubl


# Extraction des propriétés (Feature Extraction)
Ceci est la fonction qui nous permet d'extraire les propriétés pour chaque phrase (feature)

In [3]:
def extract_features(tagged_sentence, index):
    token, tag = tagged_sentence[index]
    prev_token = ""
    if index > 0:
        prev_token, prev_tag = tagged_sentence[index - 1]
    is_number = False
    try:
        if float(token):
            is_number = True
    except:
        pass
    features_dict = {"token": token
        , "lower_cased_token": token.lower()
        , "prev_token": prev_token
        , "suffix1": token[-1]
        , "suffix2": token[-2:]
        , "suffix3": token[-3:]
        , "is_capitalized": token.upper() == token
        , "is_number": is_number}
    return features_dict


In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import accuracy_score

# https://scikit-learn.org/stable/modules/feature_extraction.html
# https://www.guru99.com/pos-tagging-chunking-nltk.html
# https://www.datacamp.com/tutorial/stemming-lemmatization-python
corpus = []
f = open("data/interest-original.txt", "r")
for x in f:
    corpus.append(x)
# We remove all the $$ from the list to make it cleaner
sentence_corpus = [i for i in corpus if i != "$$\n"]
#print(sentence_corpus[1])
# We tokenize our corpus into words, for each sentence
corpus_tokens = []
for i in sentence_corpus:
    corpus_tokens.append(nltk.word_tokenize(i))
#print(corpus_tokens)
# We remove all the stop-words before stemming the text
# We do a double for loop because we want to keep the structure of sentences and words intact
stop_words = set(stopwords.words("english"))
corpus_stop_words = []
for i in corpus_tokens:
    sw = []
    for e in i:
       if e.casefold() not in stop_words:
           sw.append(e)
    corpus_stop_words.append(sw)
#print(corpus_stop_words)
# We make the stemming of all the tokens
# We do a double for loop because we want to keep the structure of sentences and words intact
corpus_stemmed = []
for i in corpus_stop_words:
    stemming = []
    for e in i:
        stemming.append(porter.stem(e))
    corpus_stemmed.append(stemming)
# We do this list for the vectorizers, because they need a list of sentences
sent_corpus_stemmed = []
for i in corpus_stemmed:
    sent_corpus_stemmed.append(" ".join(i))
#print(corpus_stemmed)
# We make the POS tagging
# corpus_tag = []
# for i in corpus_stemmed:
#     corpus_tag.append(nltk.pos_tag(i))
corpus_tag = nltk.pos_tag_sents(corpus_stemmed)
#print(corpus_tag)
# We apply the grammar on the text, so we can get the structure. We get trees for each sentence
# TODO verify if this grammar is enough for our text
grammar = "NP: {<DT>?<JJ>*<NN>}"
cp = nltk.RegexpParser(grammar)
result = []
for i in corpus_tag:
    result.append(cp.parse(i))
#print(result)
result[0].draw()    # It will draw the pattern graphically which can be seen in Noun Phrase chunking

Ici on ne fait que tester si le corpus est bien taggé et que la fonction d'extraction fonctionne bien

In [5]:
extract_features(corpus_tag[0], 1)

{'token': 'money-market',
 'lower_cased_token': 'money-market',
 'prev_token': 'yield',
 'suffix1': 't',
 'suffix2': 'et',
 'suffix3': 'ket',
 'is_capitalized': False,
 'is_number': False}

# Datasets
Ici nous faisons les jeux de données pour nos algorithmes d'apprentissage. Nous allons prendre 80% des données pour l'ensemble d'apprentissage et nous allons garder 20% pour les données de test.

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html
# Training dataset for X (80%)
X_features = []
floor = (len(corpus_tag)*0.8).__floor__()
ceiling = (len(corpus_tag)*0.8).__ceil__()
for sentence in corpus_tag[0:floor]:
    for k in range(len(sentence)):
        X_features.append(extract_features(sentence, k))

# We are using this vectorizer because our feature extractor returns a dictionary
vectoriser = DictVectorizer(sparse=False)
X_train = vectoriser.fit_transform(X_features)
print(f"X_train shape : {X_train.shape}")

# Testing dataset for X (20%)
X_features = []
for sentence in corpus_tag[ceiling:]:
    for k in range(len(sentence)):
        X_features.append(extract_features(sentence, k))

# Important: We only want to transform the data, since we need the same number of features
X_test = vectoriser.transform(X_features)
print(f"X_test shape : {X_test.shape}")

# Training labels for X_train (80%)
Y_train = []
for sentence in corpus_tag[0:floor]:
    for k in range(len(sentence)):
        Y_train.append(sentence[k][1])
print(f"Length of Y_train : {len(Y_train)}")

# Training labels for X_test (20%)
Y_test = []
for sentence in corpus_tag[ceiling:]:
    for k in range(len(sentence)):
        Y_test.append(sentence[k][1])
print(f"Length of Y_test : {len(Y_test)}")



In [6]:
X_features = []
for sentence in corpus_tag:
    for k in range(len(sentence)):
        X_features.append(extract_features(sentence, k))

vectoriser = DictVectorizer(sparse=False)
X = vectoriser.fit_transform(X_features)

y = []
for sentence in corpus_tag:
    for k in range(len(sentence)):
        y.append(sentence[k][1])

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print(f"X_train shape : {X_train.shape}")
print(f"X_test shape : {X_test.shape}")
print(f"Length of Y_train : {len(Y_train)}")
print(f"Length of Y_test : {len(Y_test)}")

X_train shape : (39027, 20316)
X_test shape : (9757, 20316)
Length of Y_train : 39027
Length of Y_test : 9757


# Naive Bayes

In [7]:
# Naive Bayes Train
clf = MultinomialNB()
clf.fit(X_train, Y_train)
# Naive Bayes predictions
predictions = clf.predict(X_test)
# 0.8000613936355264

print(accuracy_score(y_true=Y_test, y_pred=predictions))

0.7842574561853028


# Arbre de Décision

In [ ]:
params = {"criterion": ["gini"]
        , "splitter": ["best"]
        , "max_depth": [5]
        , "min_samples_split": [2, 5, 10]
          }

dt = DecisionTreeClassifier()
cv = GridSearchCV(estimator=dt, param_grid = params)
cv.fit(X_train, Y_train)

# Best parameter set
print('Best parameters found:\n', cv.best_params_)

# All results
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

C:\Users\WickkaWizz\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [15]:
# https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
# Decision tree classifier with gini for the cost function
# Trop de noeuds et de dimensions, je dois limiter la profondeur pour eviter le sur-apprentissage
dt = DecisionTreeClassifier(criterion="gini", splitter="best", max_depth=20, min_samples_split=5)
dt.fit(X_train, Y_train)
predictions = dt.predict(X_test)
print(f"Accuracy of Decision Tree with gini: {accuracy_score(y_true=Y_test, y_pred=predictions)}")
# Accuracy of Decision Tree with gini: 0.6826893512350107

dt = DecisionTreeClassifier(criterion="entropy", splitter="best", max_depth=20, min_samples_split=5)
dt.fit(X_train, Y_train)
predictions = dt.predict(X_test)
print(f"Accuracy of Decision Tree with entropy: {accuracy_score(y_true=Y_test, y_pred=predictions)}")
# Accuracy of Decision Tree with entropy: 0.7336271394895972

Accuracy of Decision Tree with gini: 0.7336271394895972


# Foret Aléatoire

In [16]:
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
rf = RandomForestClassifier(n_estimators=50, criterion="gini", max_depth=20, min_samples_split=5)
rf.fit(X_train, Y_train)
predictions = rf.predict(X_test)
print(f"Accuracy of Random Forest Tree with gini: {accuracy_score(y_true=Y_test, y_pred=predictions)}")

rf = RandomForestClassifier(n_estimators=50, criterion="entropy", max_depth=20, min_samples_split=5)
rf.fit(X_train, Y_train)
predictions = rf.predict(X_test)
print(f"Accuracy of Random Forest Tree with entropy: {accuracy_score(y_true=Y_test, y_pred=predictions)}")
# Accuracy of Random Forest Tree with gini: 0.625909603361689
# Accuracy of Random Forest Tree with entropy: 0.6430255201393871

Accuracy of Random Forest Tree with gini: 0.625909603361689
Accuracy of Random Forest Tree with entropy: 0.6430255201393871


# SVM

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
# Les données ne sont pas linéairement séparables, donc on doit imposer une limite pour les itérations pour éviter que les SVM ne se termine jamais
svm = SVC(C=1, kernel='linear', max_iter=1)
svm.fit(X_train, Y_train)
predictions = svm.predict(X_test)
print(f"Accuracy of Support Vector Machine with linear kernel: {accuracy_score(y_true=Y_test, y_pred=predictions)}")

# svm = SVC(C=1, kernel='poly', max_iter=100)
# svm.fit(X_train, Y_train)
# predictions = svm.predict(X_test)
# print(f"Accuracy of Support Vector Machine with polynomial kernel: {accuracy_score(y_true=Y_test, y_pred=predictions)}")
#
# svm = SVC(C=1, kernel='sigmoid', max_iter=100)
# svm.fit(X_train, Y_train)
# predictions = svm.predict(X_test)
# print(f"Accuracy of Support Vector Machine with sigmoid kernel: {accuracy_score(y_true=Y_test, y_pred=predictions)}")

# Multi layer perceptron (MLP)

In [ ]:
parameter_space = {
    'hidden_layer_sizes': [(50, 50, 50), (50, 100, 50), (100,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05, 0.1],
    'learning_rate': ['constant', 'adaptive'],
    'max_iter': [100, 200, 500]
}

mlp = MLPClassifier()
clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
clf.fit(X_train[0:1000, :], Y_train[0:1000])

# Best parameter set
print('Best parameters found:\n', clf.best_params_)

# All results
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html
mlp = MLPClassifier(hidden_layer_sizes=(50, 50, 50), activation='tanh')
mlp.fit(X_train, Y_train)
predictions = mlp.predict(X_test)
print(f"Accuracy of Multi Level Perceptron with activation tanh : {accuracy_score(y_true=Y_test, y_pred=predictions)}")


In [ ]:
# https://www.nltk.org/howto/chunk.html
# Le texte a déjà été chunk et pos tag
from nltk import tagstr2tree

gold_chunked_text = tagstr2tree(sentence_corpus[1])
print(gold_chunked_text)
gold_chunked_text.draw()
#[nltk.tag.str2tuple(t) for t in gold_chunked_text]
# print(tagged_token)

# Count Vectorizer

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# https://www.nltk.org/book/ch06.html
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
# https://medium.com/@eiki1212/natural-language-processing-naive-bayes-classification-in-python-e934365cf40c
# https://nlp1000.wordpress.com/2016/12/19/pos-tagging-scikit-learn/
# ngram_range=(1,2) I might use this later, but I'm not too sure I need it right now
vectorizer = CountVectorizer(input="content", analyzer='word')
vectorizer.fit_transform(sent_corpus_stemmed)
print(sent_corpus_stemmed[0])
print(vectorizer.get_feature_names_out()[0])
X_train, X_test, y_train, y_test = train_test_split(sent_corpus_stemmed, vectorizer.get_feature_names_out(), test_size=0.2, random_state=42)
X_count = vectorizer.fit_transform(X_train)

tfvectorizer = TfidfTransformer()
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_count)

# Create model(naive bayes) and training.
clf = MultinomialNB()
clf.fit(X_train_tfidf, y_train)
predictions = clf.predict(X_test)

print(accuracy_score(y_true=y_test, y_pred=predictions))


print(vectorizer.get_feature_names_out())
print(X_count.toarray())

# TF-IDF Vectorizer

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer
Tfvectorizer = TfidfVectorizer(input="content", analyzer='word')
X_tfidf = vectorizer.fit_transform(sent_corpus_stemmed)
print(vectorizer.get_feature_names_out())
print(X_tfidf.shape)
print(X_tfidf.toarray())

# Stop words

In [ ]:
# https://realpython.com/nltk-nlp-python/#:~:text=Natural%20language%20processing%20(NLP)%20is,and%20contains%20human%2Dreadable%20text
# https://stackoverflow.com/questions/52712254/how-to-eliminate-stop-words-only-using-scikit-learn
stop_words = set(stopwords.words("english"))
filtered_list = [word for word in words_in_quote if word.casefold() not in stop_words]